In [ ]:
### 使用提示词完成任务

In [5]:
# 导入依赖库
import dashscope
import os

# 从环境变量中获取 API Key
dashscope.api_key = os.getenv('DASHSCOPE_API_KEY')

# 基于 prompt 生成文本
# 使用 deepseek-v3 模型
def get_completion(prompt, model="deepseek-v3"):
    messages = [{"role": "user", "content": prompt}]    # 将 prompt 作为用户输入
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format='message',  # 将输出设置为message形式
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content  # 返回模型生成的文本
    
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
# 目标
{instruction}

# 用户输入
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# 调用大模型
response = get_completion(prompt)
print(response)

==== Prompt ====

# 目标

你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。


# 用户输入

办个100G的套餐。


# 分析结果

根据用户输入"办个100G的套餐"，可以识别出以下选择条件：

1. **月流量需求**：用户明确要求100GB的月流量
   - 识别依据：直接提到"100G"

2. **其他属性未明确**：
   - 名称：未提及特定套餐名称
   - 月费价格：未提及价格要求

因此，用户的主要选择标准是月流量达到100GB，而对套餐名称和价格没有提出具体要求。


### JSON格式

In [9]:
# 输出格式
output_format = """
以 JSON 格式输出
"""

# 稍微调整下咒语，加入输出格式
prompt = f"""
# 目标
{instruction}

# 输出格式
{output_format}

# 用户输入
{input_text}
"""

# 调用大模型，指定用 JSON mode 输出
response = get_completion(prompt)
print(response)

```json
{
  "提示词": "我需要办理一个100G流量的手机套餐，请提供适合的套餐选项，包括价格、包含的服务（如通话分钟、短信数量等）、合约期限以及是否有任何优惠活动。",
  "优化建议": "原提示词过于简略，缺乏具体需求细节（如预算、是否需要国际服务、对运营商有无偏好等）。优化后的提示词明确要求了套餐的核心要素（流量、价格、附加服务）和潜在优惠信息，便于服务方提供精准方案。建议进一步补充使用场景（如个人/家庭使用）和网络需求（如5G优先）以获取更匹配的推荐。",
  "问题示例": [
    "您对运营商的网络覆盖或服务质量有特定要求吗？",
    "这个套餐是用于个人使用还是多人共享？",
    "您是否需要包含国际漫游或特定定向流量（如视频/社交软件免流）？"
  ]
}
```


### CoT示例

In [7]:
instruction = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

#cot = ""
cot = "请一步一步分析对话"

prompt = f"""
# 目标
{instruction}
{cot}

# 输出格式
{output_format}

# 对话上下文
{context}
"""

response = get_completion(prompt)
print(response)

让我们一步一步分析客服的回答是否符合规范：

1. 礼貌性：使用"亲"开头，虽然试图表现亲切，但不符合"官方口吻"的要求（网络用语倾向） → 不符合
2. 官方口吻：使用"亲"这种网络用语 → 不符合
3. 信息准确性：
   - 产品名称：提及"无限套餐" → 正确
   - 月费价格：提及"300元" → 正确
   - 月流量：提及"1000G" → 正确
   → 信息准确
4. 话题终结者：使用问句"您感兴趣吗？"保持对话开放性 → 符合

主要问题在于使用了网络用语"亲"，不符合官方口吻的要求。

最终判断：N


### 使用Prompt调优Prompt

In [8]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
# 目标
{instruction}

# 用户提示词
{user_prompt}
"""

response = get_completion(prompt)
print(response)

# 提示词

你是一名专业的手机流量套餐客服代表，名叫小瓜。你的任务是帮助用户选择最适合他们需求的流量套餐。请按照以下步骤与用户互动：

1. 首先热情问候用户："您好！我是流量套餐顾问小瓜，很高兴为您服务。"
2. 询问用户的基本需求：
   - 每月大概需要多少流量？
   - 是否有预算限制？
   - 是否是学生(可享受校园套餐)？
3. 根据用户需求推荐1-2个最合适的套餐选项，并简要说明推荐理由
4. 提供套餐详情：
   - 经济套餐：月费50元，10G流量
   - 畅游套餐：月费180元，100G流量
   - 无限套餐：月费300元，1000G流量
   - 校园套餐：月费150元，200G流量(仅限在校生)
5. 询问用户是否有其他问题或需要进一步帮助
6. 结束时说："感谢您选择小瓜的服务，祝您使用愉快！"

请保持专业、友好且简洁的沟通风格，避免使用过于复杂的术语。

# 优化建议

当前提示词可以改进的方面：
1. 互动流程可以更自然：建议将询问需求的过程设计为更对话式的交流，而非机械的问卷式提问
2. 增加个性化元素：可以加入根据用户使用习惯(如视频观看频率、游戏需求等)的评估
3. 明确边界条件：应说明哪些情况需要转接人工客服(如特殊优惠申请)
4. 增加比较功能：可以提供不同套餐间的横向对比表格
5. 加入促销信息：如果有新用户优惠或合约优惠，应该包含这些信息

# 问题示例

1. 您希望这个客服AI主要面向哪类用户群体？(如年轻人、商务人士等)
2. 是否需要加入套餐变更或退订流程的指导？
3. 是否允许客服代表根据用户反馈调整推荐策略？(如用户觉得推荐不合适时)
